In [0]:
# Hugging Face의 트랜스포머 모델을 설치
!pip install transformers
!pip install pytorch_pretrained_bert==0.4.0
import tensorflow as tf
import torch
from pytorch_pretrained_bert import BertForSequenceClassification
from transformers import AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import random
import time
import datetime

#드라이브에 마운트
from google.colab import drive
drive.mount('/content/drive')

#기본 BERT모델의 tokenizer를 ETRI 한국어 버트 모델의 tokenizer으로 변경함.
#한국어 버트 모델 다운로드를 위해서 ETRI의 승인을 받아야 합니다.
!cp "/content/drive/My Drive/bert_pytorch/001_bert_morp_pytorch/src_tokenizer/tokenization_morp.py" /usr/local/lib/python3.6/dist-packages/pytorch_pretrained_bert/
from pytorch_pretrained_bert.tokenization_morp import BertTokenizer

# 데이터 로드 및 *전처리*

In [0]:
# 판다스로 훈련셋 데이터 로드
test_data = pd.read_csv("/content/drive/My Drive/하/test_morphs.txt", sep='\t',header=None)
train_data=pd.read_csv('/content/drive/My Drive/하/train_morphs.txt', sep='\t',header=None)
print(train_data)
print(test_data)

               0                                                  1  2
0        9976970  아/IC 더/NNP 빙/NNG ../SE 진짜/MAG 짜증/NNG 나/VV 네/XS...  0
1        3819312  흠/IC ./SF ../SE 포스터/NNG 보/VV 고/EC 초딩/NNG 영화/NN...  1
2       10265843  너무/MAG 재/VA 밓/VV 었/EP 다/EC 그래서/MAJ 보/VV 는/ETM ...  0
3        9045019  교도소/NNG 이야기/NNG 이/VCP 구/EF 먼/EC ../SE 솔직히/MAG ...  0
4        6483659  사이/NNP 몬/NNG 페그/NNP 의/JKG 익살/NNG 스럽/XSA ㄴ/ETM ...  1
...          ...                                                ... ..
149995   6222902  인간/NNG 이/JKS 문제지/NNG ./SF ./SE 소/NNG 는/JX 뭔/MM...  0
149996   8549745             평점/NNG 이/JKS 너무/MAG 낮/VA 아서/EC .../SE   1
149997   9311800  이것/NP 이/JKS 뭐/NP 이/VCP 오/EF ?/SF 한국인/NNG 은/JX ...  0
149998   2376369  청춘/NNG 영화/NNG 의/JKG 최고봉/NNG ./SP 방황/NNG 과/JKB ...  1
149999   9619869  한국/NNP 영화/NNG 최초/NNG 로/JKB 수간/NNG 하/XSV 는/ETM ...  0

[150000 rows x 3 columns]
             0                                                  1  2
0      6270596                                      

In [0]:
#train set 리뷰 문장 추출
sentences_train=list()
for i in range(len(train_data[1])):
  if(type(train_data[1][i])==float):
    sentences_train.append("[CLS] [SEP]")
  else:
    sentences_train.append("[CLS] "+train_data[1][i]+" [SEP]")
  
#test set 리뷰 문장 추출
sentences_test=list()
for i in range(len(test_data[1])):
  if(type(test_data[1][i])==float):
    sentences_test.append("[CLS] [SEP]")
  else:

    sentences_test.append("[CLS] "+test_data[1][i]+" [SEP]")

In [0]:
print(sentences_train[0:9])
print(len(sentences_train))
print(sentences_test[0:9])
print(len(sentences_test))


['[CLS] 아/IC 더/NNP 빙/NNG ../SE 진짜/MAG 짜증/NNG 나/VV 네/XSN 요목/NNG 소리/NNG  [SEP]', '[CLS] 흠/IC ./SF ../SE 포스터/NNG 보/VV 고/EC 초딩/NNG 영화/NNG 줄/NNG ./SF .../SE 오버/NNG 연기/NNG 조차/JX 가볍/VA 지/EC 않/VX 구나/EC  [SEP]', '[CLS] 너무/MAG 재/VA 밓/VV 었/EP 다/EC 그래서/MAJ 보/VV 는/ETM 것/NNB 을/JKO 추천/NNG 하/XSV ㄴ다/EC  [SEP]', '[CLS] 교도소/NNG 이야기/NNG 이/VCP 구/EF 먼/EC ../SE 솔직히/MAG 재미/NNG 는/JX 없/VA 다/EF ./SF ./SE 평점/NNG 조정/NNG  [SEP]', '[CLS] 사이/NNP 몬/NNG 페그/NNP 의/JKG 익살/NNG 스럽/XSA ㄴ/ETM 연기/NNG 가/JKS 돋보이/VV 었/EP 던/ETM 영화/NNG !/SP 스파/NNG 이더/NNP 맨/NNG 에서/JKB 늙/VV 어/EC 보이/VV 기/ETN 만/JX 하/VX 였/EP 던/ETM 커스틴/NNP 던/NNP 스트/NNG 가/JKS 너무나/MAG 도/JX 이쁘/VV 어/EC 보이/VV 었/EP 다/EC  [SEP]', '[CLS] 막걸음마/NNG 떼/VV ㄴ/ETM 3/SN 세/NNB 부터/JX 초등학교/NNG 1/SN 학년/NNG 생/XSN 이/VCP ㄴ/ETM 8/SN 살/NNB 용/NNG 영화/NNG ./SP ㅋ/IC ㅋ/NNG ㅋ/IC .../SE 별반/NNG 개/NNG 도/JX 아깝/VA ㅁ/ETN ./SF  [SEP]', '[CLS] 원작/NNG 의/JKG 긴장감/NNG 을/JKO 제대로/MAG 살리/VV 어/EC 내/VX 지/EC 못하/VX 였/EP 다/EF ./SF  [SEP]', '[CLS] 별반/MAG 개/NNG 도/JX 아깝/VA 다/EC 욕/NNG 나오/VV ㄴ/ETM 다/MAG 이/MM 응/NNG 경길용우/NNP 연기

In [0]:
# train set 라벨 추출
labels_train = list()
for i in range(len(train_data[2])):
  labels_train.append(train_data[2][i])
print(labels_train)
print(len(labels_train))
print(labels_train[0:9])

# test set 라벨 추출
labels_test = list()
for i in range(len(test_data[2])):
  labels_test.append(test_data[2][i])
print(labels_test)
print(len(labels_test))
print(labels_test[0:9])


#메모리 최적화를 위해 필요 없는 변수 삭제
del(train_data)
del(test_data)


In [0]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained("/content/drive/My Drive/bert_pytorch/001_bert_morp_pytorch/vocab.korean_morp.list", do_lower_case=False)
tokenized_train_texts = [tokenizer.tokenize(sent) for sent in sentences_train]
print (sentences_train[0])
print (tokenized_train_texts[0])
print(len(tokenized_train_texts))

# BERT의 토크나이저로 문장을 토큰으로 분리
tokenized_test_texts = [tokenizer.tokenize(sent) for sent in sentences_test]
print (sentences_test[0])
print (tokenized_test_texts[0])
print(len(tokenized_test_texts))


#메모리 최적화를 위해 필요 없는 변수 삭제
del(sentences_train)
del(sentences_test)

[CLS] 아/IC 더/NNP 빙/NNG ../SE 진짜/MAG 짜증/NNG 나/VV 네/XSN 요목/NNG 소리/NNG  [SEP]
['[CLS]', '_', '아/IC_', '더/NNP_', '빙/NNG_', '.', '.', '/SE_', '진', '짜', '/MAG_', '짜', '증/NNG_', '나/VV_', '네/XSN_', '요', '목/NNG_', '소리/NNG_', '[SEP]', '_']
150000
[CLS] 굳/MAG ㅋ/NNG  [SEP]
['[CLS]', '_', '굳', '/MAG_', 'ㅋ', '/NNG_', '[SEP]', '_']
50000


In [0]:
#필요 없는 띄어쓰기 삭제
_removed_tokenized_train_texts=list()
for sent in tokenized_train_texts:
  while('_' in sent):
    sent.remove('_')
  _removed_tokenized_train_texts.append(sent)

#필요 없는 띄어쓰기 삭제
_removed_tokenized_test_texts=list()
for sent in tokenized_test_texts:
  while('_' in sent):
    sent.remove('_')
  _removed_tokenized_test_texts.append(sent)


#메모리 최적화를 위해 필요 없는 변수 삭제
del(tokenized_train_texts)
del(tokenized_test_texts)

In [0]:
print(_removed_tokenized_train_texts[0])
print(tokenizer.convert_tokens_to_ids(_removed_tokenized_train_texts[0]))
print(len(_removed_tokenized_train_texts))

['[CLS]', '아/IC_', '더/NNP_', '빙/NNG_', '.', '.', '/SE_', '진', '짜', '/MAG_', '짜', '증/NNG_', '나/VV_', '네/XSN_', '요', '목/NNG_', '소리/NNG_', '[SEP]']
[2, 7016, 4600, 3488, 18168, 18168, 21245, 351, 3349, 228, 3349, 2315, 515, 4384, 826, 1040, 1557, 3]
150000


In [0]:
print(_removed_tokenized_test_texts[0])
print(tokenizer.convert_tokens_to_ids(_removed_tokenized_test_texts[0]))
print(len(_removed_tokenized_test_texts))

['[CLS]', '굳', '/MAG_', 'ㅋ', '/NNG_', '[SEP]']
[2, 6956, 228, 10494, 83, 3]
50000


In [0]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 256

# 토큰을 숫자 인덱스로 변환
input_train_ids = [tokenizer.convert_tokens_to_ids(x) for x in _removed_tokenized_train_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_train_ids = pad_sequences(input_train_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 256

# 토큰을 숫자 인덱스로 변환
input_test_ids = [tokenizer.convert_tokens_to_ids(x) for x in _removed_tokenized_test_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_test_ids = pad_sequences(input_test_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")


#필요 없는 변수 삭제
del(_removed_tokenized_train_texts)
del(_removed_tokenized_test_texts)


In [0]:
print(input_train_ids[0])
len(input_train_ids)

[    2  7016  4600  3488 18168 18168 21245   351  3349   228  3349  2315
   515  4384   826  1040  1557     3     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

150000

In [0]:
print(input_test_ids[0])
len(input_test_ids)

[    2  6956   228 10494    83     3     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

50000

In [0]:
# 어텐션 마스크 초기화
attention_train_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_train_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_train_masks.append(seq_mask)

print(attention_train_masks[0])
print(len(attention_train_masks))
# 어텐션 마스크 초기화
attention_test_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_test_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_test_masks.append(seq_mask)

print(attention_test_masks[0])
print(len(attention_test_masks))

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [0]:
# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(input_train_ids)
train_labels = torch.tensor(labels_train)
train_masks = torch.tensor(attention_train_masks)

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(len(train_inputs))
print(len(train_labels))
print(len(train_masks))


# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_test_ids)
test_labels = torch.tensor(labels_test)
test_masks = torch.tensor(attention_test_masks)

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(len(train_inputs))
print(len(train_labels))
print(len(train_masks))


print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])
print(len(test_inputs))
print(len(test_labels))
print(len(test_masks))

tensor([    2,  7016,  4600,  3488, 18168, 18168, 21245,   351,  3349,   228,
         3349,  2315,   515,  4384,   826,  1040,  1557,     3,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [0]:
# 배치 사이즈
batch_size = 16

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

# 디바이스 설정 및 모델 생성


In [0]:
import tensorflow as tf

# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [0]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [0]:
# 분류를 위한 BERT 모델 생성
# 미리 학습된 한국어 BERT 모델을 사용함.
model = BertForSequenceClassification.from_pretrained("/content/drive/My Drive/bert_pytorch/001_bert_morp_pytorch/", num_labels=2)
model.cuda()

In [0]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 2
epoch=0
# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 학습률을 조금씩 감소시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

# 학습 시작


In [0]:
#랜덤 시드 설정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):

    #학습 시작
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, 2))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            print(total_loss/step)

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs
        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.12f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
    #매 에포크 마다 결과값 저장
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler' : scheduler.state_dict()
        }, "/content/drive/My Drive/kor_bert_senti"+"%d"%(epoch_i))
print("")
print("Training complete!")


======== Epoch 1 / 2 ========
Training...
  Batch   500  of  9,375.    Elapsed: 0:03:56.
0.42218369123339655
  Batch 1,000  of  9,375.    Elapsed: 0:07:51.
0.3885974925868213
  Batch 1,500  of  9,375.    Elapsed: 0:11:46.
0.36613654176642496
  Batch 2,000  of  9,375.    Elapsed: 0:15:41.
0.3543316984605044
  Batch 2,500  of  9,375.    Elapsed: 0:19:36.
0.3453534990131855
  Batch 3,000  of  9,375.    Elapsed: 0:23:32.
0.33892503745357194
  Batch 3,500  of  9,375.    Elapsed: 0:27:28.
0.33290335967817475
  Batch 4,000  of  9,375.    Elapsed: 0:31:25.
0.3285522306282073
  Batch 4,500  of  9,375.    Elapsed: 0:35:21.
0.32392874596226545
  Batch 5,000  of  9,375.    Elapsed: 0:39:18.
0.3193317886274308
  Batch 5,500  of  9,375.    Elapsed: 0:43:14.
0.3164164243370972
  Batch 6,000  of  9,375.    Elapsed: 0:47:11.
0.3125434384603674
  Batch 6,500  of  9,375.    Elapsed: 0:51:08.
0.30932538038750107
  Batch 7,000  of  9,375.    Elapsed: 0:55:05.
0.30595358233393305
  Batch 7,500  of  9,375. 

In [0]:
# 그래디언트 초기화
model.zero_grad()
print("")
print("Running Validation...")
#시작 시간 설정
t0 = time.time()
# 평가모드로 변경
model.eval()
# 데이터로더에서 배치만큼 반복하여 가져옴
for batch in test_dataloader:
    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs
    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1
print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


Running Validation...
  Accuracy: 0.89
  Validation took: 0:07:22

Training complete!
